In [1]:
import os.path

import pandas as pd
import glob
import os

files = glob.glob('C:/Users/liams/Documents/capstone gis data/data/processed/tree_species_stand_density' + '/*.csv')

dataframes = []

for path in files:
    df = pd.read_csv(path).rename(columns={'norm_value': os.path.basename(path)})
    df['long'] = df['long'].round(4)
    df['lat'] = df['lat'].round(4)
    df = df.set_index(['long', 'lat'])
    dataframes.append(df)

frame = pd.concat(dataframes, axis=1).fillna(0)

In [2]:
frame

,,WA_pacific_yew_basal_area.csv,WA_red_alder_stand_density.csv,WA_sitka_spruce_stand_density.csv,WA_western_hemlock_stand_density.csv
long,lat,,,,
-121.4029,48.9996,0.000000,0.0,0.0,0.361914
-121.4026,48.9996,0.012658,0.0,0.0,0.204387
-121.4023,48.9996,0.012658,0.0,0.0,0.204387
-121.4021,48.9996,0.069620,0.0,0.0,0.156530
-121.4034,48.9994,0.006329,0.0,0.0,0.290130
...,...,...,...,...,...
-121.4428,45.5719,0.000000,0.0,0.0,0.093719
-121.4325,45.5719,0.000000,0.0,0.0,0.046859
-121.4323,45.5719,0.000000,0.0,0.0,0.154536


In [13]:
import hashlib

hash = hashlib.md5()
hash.update('-121.4029, 48.9996'.encode('utf8'))
str(hash.hexdigest())

'cec0ee69f13239de668225118e227e85'

In [17]:
index_list = frame.index.tolist()

In [18]:
index_list[0]

(-121.4029, 48.9996)